# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [1]:
!pip install python-dotenv
!pip install jupyter_bokeh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 56.3 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [2]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [3]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [4]:
#Definition of the tables.
import pandas as pd

# Table and definitions sample
data = {'table':['employees','salary','studies'], # ENTER A TABLE COLUMNS HERE,
        'definition':["Employee information, name...","Salary details for each year","Educational studies, name of the institution, ..."]} # ENTER A TABLE DEFINITATIONS HERE
df = pd.DataFrame(data)
print(df)

       table                                         definition
0  employees                      Employee information, name...
1     salary                       Salary details for each year
2    studies  Educational studies, name of the institution, ...


In [5]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [6]:
print(text_tables)

employees: Employee information, name...
salary: Salary details for each year
studies: Educational studies, name of the institution, ...


In [7]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [8]:
# Creating the prompt with the user question and table definitions

query = "Which tables contain employee salary details?"
pqt1 = prompt_question_tables.format(tables=text_tables, question=query)


In [9]:
print(return_OAI(pqt1))

```json
{
    "tables": ["employees", "salary"]
}
```


In [10]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question=  "Please provide the tables that are related to the query and include their definitions.?"
)

In [11]:
print(return_OAI(pqt3))

```json
{
    "tables": [
        {
            "name": "employees",
            "definition": "Employee information, including name, employee ID, department, position, hire date, etc."
        },
        {
            "name": "salary",
            "definition": "Salary details for each year, including employee ID, salary amount, year, bonus, etc."
        },
        {
            "name": "studies",
            "definition": "Educational studies information, including employee ID, institution name, degree obtained, graduation year, etc."
        }
    ]
}
```


# Exercise
 - Complete the prompts similar to what we did in class.
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

   - Formulating the question immediately is important: The way the question is formulated significantly impacts the quality of the answer. Clear, straightforward questions yield more accurate answers, while general or complex questions yield more general answers.
Handling Multiple Queries: When using queries on multiple attributes (such as education and salary), GPT performs well as long as the question is not ambiguous or open-ended.



In [15]:
import pandas as pd
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv


# Test with different queries to check the model's response

# Query 1: Request to identify tables containing employee salary details
query_1 = "Which tables provide detailed information about employee salary?"
pqt1 = prompt_question_tables.format(tables=text_tables, question=query_1)
response_1 = return_OAI(pqt1)
print("Response to Query 1:", response_1)

# Query 2: Ask for tables relevant to the provided question along with their definitions
query_2 = "Could you list the tables that are relevant to the given question, along with their definitions?"
pqt2 = prompt_question_tables.format(tables=text_tables, question=query_2)
response_2 = return_OAI(pqt2)
print("Response to Query 2:", response_2)

# Experiment with a more creative and complex query
query_3 = "Please identify which tables are related to both educational background and salary information, and provide details."
pqt3 = prompt_question_tables.format(tables=text_tables, question=query_3)
response_3 = return_OAI(pqt3)
print("Response to Query 3:", response_3)



Response to Query 1: ```json
{
    "tables": ["employees", "salary"]
}
```
Response to Query 2: ```json
{
    "tables": {
        "employees": "Employee information, name, department, position, etc.",
        "salary": "Salary details for each year, including amount, bonuses, and deductions.",
        "studies": "Educational studies, including the name of the institution and degree obtained."
    }
}
```
Response to Query 3: ```json
{
    "tables": ["employees", "salary", "studies"]
}
```
